### Visualización

In [5]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import datetime as dt

import folium 
from folium.plugins import HeatMap

In [7]:
df = pd.read_csv('./iFlowDatos_Limpios.csv')
df.head()

,iddomicilioorden,direccion,provincia,localidad,InicioHorario1,FinHorario1,latitud,longitud,cliente,mes,Bultos,Peso,Unidades,VisitaPlanificadaFecha,VisitaPlanificadaHora,VisitaFecha,VisitaHora_Inicio,VisitaHora_Fin
0,74956,VIDAL 2044,Ciudad Autónoma de Buenos Aires,Belgrano,901,1401,-34.564779,-58.459956,70,Mayo,2.516667,24.910,30,2024-05-23,12:06:00,2024-05-23,14:13:53,14:14:53
1,74956,VIDAL 2044,Ciudad Autónoma de Buenos Aires,Belgrano,901,1401,-34.564779,-58.459956,70,Mayo,2.866667,26.260,38,2024-05-10,12:05:00,2024-05-10,16:43:00,16:43:00
2,74956,VIDAL 2044,Ciudad Autónoma de Buenos Aires,Belgrano,901,1401,-34.564779,-58.459956,70,Mayo,2.400000,24.888,32,2024-05-30,12:03:00,2024-05-30,09:41:36,09:50:08
3,74956,VIDAL 2044,Ciudad Autónoma de Buenos Aires,Belgrano,901,1401,-34.564779,-58.459956,70,Junio,1.800000,14.040,28,2024-06-13,12:14:00,2024-06-13,13:32:55,13:39:55
4,74956,VIDAL 2044,Ciudad Autónoma de Buenos Aires,Belgrano,901,1401,-34.564779,-58.459956,70,Junio,2.116667,14.377,31,2024-06-27,12:27:00,2024-06-27,15:36:00,15:36:00


### Análisis Univariado

### Análisis Bivariado

### Mapas Interactivos

In [8]:
df = df.dropna(subset=['latitud', 'longitud'])

# Agrupar por localidad y calcular la cantidad total de repartos y las coordenadas promedio
repartos_por_localidad = df.groupby('localidad').agg({
        'latitud': 'mean',
        'longitud': 'mean',
        'localidad': 'size'
    }).rename(columns={'localidad': 'cantidad_repartos'}).reset_index()

# Crear el mapa centrado en Buenos Aires
m = folium.Map(location = [-34.603722, -58.381592], zoom_start = 12)

# Añadir un círculo por localidad con información de repartos
for i, row in repartos_por_localidad.iterrows():
    folium.CircleMarker(
        location    = [row['latitud'], row['longitud']],
        radius      = np.log(row['cantidad_repartos']) * 5,  # Ajustar el tamaño del círculo
        popup       = f"Localidad: {row['localidad']}<br>Cantidad de repartos: {row['cantidad_repartos']}",
        color       = 'blue',
        fill        = True,
        fill_color  = 'blue'
    ).add_to(m)

# Añadir un HeatMap basado en las coordenadas promedio de cada localidad
heat_data = [[row['latitud'], row['longitud'], row['cantidad_repartos']] for index, row in repartos_por_localidad.iterrows()]
HeatMap(heat_data).add_to(m)

# Guardar el mapa en un archivo HTML
m.save("heatmap_por_localidad_buenos_aires.html")

# Mostrar el mapa (si estás en Jupyter Notebook)
m